In [21]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

O **Naive Bayes é um algoritmo de classificação probabilística que se baseia no Teorema de Bayes ** e assume uma independência ingênua (daí o "naive" em seu nome) entre os recursos (palavras, no caso de textos) para simplificar o cálculo das probabilidades. Ele é amplamente usado em tarefas de processamento de linguagem natural, como classificação de texto.

**O teorema de Bayes é uma fórmula matemática que nos permite calcular a probabilidade de um evento ocorrer**, dado que outro evento já ocorreu
P(A | B) = P(B | A) * P(A) / P(B)

P(A | B) é a probabilidade do evento A ocorrer, dado que o evento B já ocorreu.

P(B | A) é a probabilidade do evento B ocorrer, dado que o evento A já ocorreu.

P(A) e P(B) são as probabilidades dos eventos A e B ocorrerem, respectivamente.

Podemos usar esse algoritimo para análisar se o conjunto de twites desse Dataset
Trata-se de mensagens sobre a mudânça climática. Para tanto separamos as mensagens em

pró - Políticas de enfrentamento as mudâças climáticas

anti - Políticas de enfrentamento as mudâças climáticas

neutro - Políticas de enfrentamento as mudâças climáticas

Ou se trata-se de uma Fake News

In [5]:
df = pd.read_csv('../input/twitter-climate-change-sentiment-dataset/twitter_sentiment_data.csv')

In [6]:
df.head(8)

,sentiment,message,tweetid
0,-1,@tiniebeany climate change is an interesting h...,792927353886371840
1,1,RT @NatGeoChannel: Watch #BeforeTheFlood right...,793124211518832641
2,1,Fabulous! Leonardo #DiCaprio's film on #climat...,793124402388832256
3,1,RT @Mick_Fanning: Just watched this amazing do...,793124635873275904
4,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125156185137153
5,0,Unamshow awache kujinga na iko global warming ...,793125429418815489
6,2,"RT @cnalive: Pranita Biswasi, a Lutheran from ...",793125430236684289
7,2,RT @CCIRiviera: Presidential Candidate #Donald...,793126558688878592


**Pré-processamento de Dados:**
Primeiro, é pré-processar os tweets. 

Para tanto iremos remover caracteres especiais, pontuações, stopwords (palavras comuns que não trazem muita informação) e a tokenização dos tweets em palavras individuais.

In [11]:
# Pré-processamento avançado com remoção de caracteres numéricos e normalização
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Converte para minúsculas
    text = text.lower()
    # Remover caracteres numéricos
    text = re.sub(r'\d+', '', text)
    # Remover URLs e emojis
    text = re.sub(r'http\S+|www.\S+|@[A-Za-z0-9]+', '', text)
    text = re.sub('[^\w\s#@/:%.,_-]', '', text, flags=re.UNICODE)
    # Remover stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Stemming
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    return ' '.join(stemmed_words)

df['message'] = df['message'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Essa primeira parte** realiza um pré-processamento avançado em um conjunto de mensagens de texto usando a biblioteca NLTK (Natural Language Toolkit) para Python.

**Para tanto a linha nltk.download faz um download de stopwords**, ou seja, palavras comuns que são geralmente removidas durante a análise de texto

Um conjunto de stopwords em inglês é carregado na variável stop_words. 
Após isso, um objeto PorterStemmer é criado, que é usado para reduzir as palavras às suas formas básicas (stems) para tratamento mais eficiente.(melhorar a performânce) 

**A partir disso eu crio uma função:
**


Esta função realiza uma série de etapas para pré-processar o texto fornecido:

Converte todo o texto para minúsculas.

Remove caracteres numéricos usando a expressão regular \d+.

Remove URLs e menções de usuário (emojis) usando expressões regulares.

Remove caracteres especiais usando a expressão regular [^\w\s#@/:%.,_-].

Divide o texto em palavras individuais.

Remove as palavras que estão presentes na lista de stopwords.

Aplica o processo de stemming a cada palavra para reduzi-las à sua forma base.


**Após a criação dessa função temos:
**


A função preprocess_text é aplicada à coluna 'message' de um DataFrame df. (por isso a demora para rodar essa parte do programa)

In [22]:
# Outras representações de palavras (TF-IDF)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['message'])

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)
# Balanceamento de classes com SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Ajuste de hiperparâmetros com GridSearchCV
params = {'alpha': [0.1, 0.5, 1.0]}
naive_bayes_model = MultinomialNB()
grid_search = GridSearchCV(naive_bayes_model, params, cv=5, scoring='accuracy')
grid_search.fit(X_train_resampled, y_train_resampled)

# Melhor configuração de hiperparâmetros
best_alpha = grid_search.best_params_['alpha']
print(f'Melhor valor de alpha encontrado: {best_alpha}')

# Treinar o modelo Naive Bayes com a melhor configuração de hiperparâmetros
naive_bayes_model = MultinomialNB(alpha=best_alpha)
naive_bayes_model.fit(X_train_resampled, y_train_resampled)
# Avaliar o modelo
y_pred = naive_bayes_model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo Naive Bayes: {accuracy:.2f}')

Melhor valor de alpha encontrado: 0.1
Acurácia do modelo Naive Bayes: 0.65


Nessa parte do programa continuamos na parte de pré processamento de dados

Para tanto a classe TfidfVectorizer do scikit-learn é usada para converter o texto pré-processado em uma representação numérica usando a **técnica TF-IDF**. cada mensagem é transformada em vetores ponderados
Esse vetores levam em consideração a frequência das palavras em cada mensagem em relação à frequência inversa nos documentos.
X agora contém as representações numéricas das mensagens, e y contém os rótulos de sentimento associados.

**Depois é feito o treinamento:**

os dados são divididos em conjuntos de treinamento e teste usando a função train_test_split do scikit-learn. 

Depois é necessário balancear as classes dentro do conjunto de treinamento. Por isso usado o Smote

Isso cria novas instâncias sintéticas das classes minoritárias para equilibrar a distribuição de classes. X_train_resampled e y_train_resampled agora contêm os dados de treinamento balanceados.

Nesta etapa, um modelo Naive Bayes é usado (especificamente MultinomialNB) e o GridSearchCV é aplicado para ajustar o hiperparâmetro alpha (que controla a suavização) usando validação cruzada. 

O parâmetro scoring é definido como 'accuracy' para avaliar o desempenho.

In [23]:
# Relatório de classificação
class_report = classification_report(y_test, y_pred)
print('Relatório de Classificação:')
print(class_report)

Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.40      0.60      0.48       784
           0       0.50      0.51      0.50      1582
           1       0.80      0.65      0.72      4514
           2       0.64      0.78      0.70      1909

    accuracy                           0.65      8789
   macro avg       0.59      0.64      0.60      8789
weighted avg       0.68      0.65      0.66      8789



**Com isso imprimimos o melhor alpha** e a partir disso novo modelo Naive Bayes é treinado usando a melhor configuração de hiperparâmetros encontrada durante o GridSearchCV

O relatório de classificação fornecerá métricas como precisão, recall e F1-score para cada classe de sentimento 
(por exemplo, "falso" ou "verdadeiro"), 

Ajudando a entender como o modelo está performando para cada classe separadamente

Temos que:

**precision (precisão):** a precisão para cada classe indica quantas das mensagens classificadas como essa classe realmente pertencem a essa classe.

**recall (revocação)**: o recall para cada classe indica quantas das mensagens reais dessa classe foram corretamente identificadas pelo modelo.

**f1-score:** O f1-score é a média harmônica entre a precisão e o recall. Ele oferece uma métrica equilibrada que leva em consideração tanto falsos positivos quanto falsos negativos. É especialmente útil quando você tem classes desbalanceadas.

**support**: O suporte indica o número de instâncias reais de cada classe no conjunto de teste.

**accuracy (acurácia):** A acurácia é a proporção total de previsões corretas em relação ao total de instâncias. No caso 0.65.

**macro avg e weighted avg:** Essas linhas fornecem médias das métricas para todas as classes. A média ponderada (weighted avg) dá mais peso às classes maiores (com base no suporte), enquanto a média não ponderada (macro avg) trata todas as classes igualmente.

Com isso:

O modelo tem desempenho variado para cada classe. Ele é mais preciso e tem recall mais alto para a classe 1, o que indica que está lidando melhor com mensagens que provavelmente contêm informações verdadeiras.
A classe -1 tem a menor precisão, o que sugere que o modelo está fazendo mais falsos positivos (classificando mensagens falsas como verdadeiras).
A classe 2 tem o recall mais alto, indicando que o modelo está identificando corretamente a maioria das mensagens reais dessa classe.